In [1]:
#Celda para librerías
import sklearn as sk


import matplotlib as plt
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.model_selection import StratifiedKFold

# LightGBM
from lightgbm import LGBMClassifier
import lightgbm as lgb



from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score


import json

import warnings
warnings.filterwarnings('ignore')

print(np.__version__)
from ydata_profiling import ProfileReport
import os
print("Directorio actual:", os.getcwd())

# Have all columns appear when dataframes are displayed.
pd.set_option('display.max_columns', None) 
# Have 100 rows appear when a dataframe is displayed
pd.set_option('display.max_rows', 500)
# Display dimensions whenever a dataframe is printed out.
pd.set_option('display.show_dimensions', True)

#Importando los datos

#app_train_def = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/application_train_preprocesado_definitivo_v1.csv')
#app_train_def_2 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v2.csv')
#app_train_def_3 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v3.csv')
#app_train_def_4 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v4.csv')
app_train_def_7 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v7.csv')
app_test = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/app_test_pruebav1.csv')
#print(app_train_def_4.info())
print(app_test.shape)
print(app_train_def_7.shape)


1.26.4
Directorio actual: /home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS
(48744, 1323)
(307511, 1323)


In [3]:
varianza = app_train_def_7.std()
print(varianza)

Unnamed: 0.1                                            88770.923652
TARGET                                                      0.272419
NAME_CONTRACT_TYPE                                          0.293509
CODE_GENDER                                                 0.474261
FLAG_OWN_CAR                                                0.473746
                                                            ...     
LATEST_CREDIT_TYPE_CAT_(BUREAU)_Unknown type of loan        0.009711
LATEST_CREDIT_ACTIVE_CAT_(BUREAU)_Active                    0.432494
LATEST_CREDIT_ACTIVE_CAT_(BUREAU)_Bad debt                  0.001803
LATEST_CREDIT_ACTIVE_CAT_(BUREAU)_Closed                    0.431233
LATEST_CREDIT_ACTIVE_CAT_(BUREAU)_Sold                      0.046418
Length: 1323, dtype: float64


In [6]:
varianza[varianza==0]

CLOSED_AMT_CREDIT_SUM_OVERDUE_TO_PROLONGED_RATIO_MEAN_(BUREAU)                               0.0
CLOSED_AMT_CREDIT_SUM_OVERDUE_TO_PROLONGED_RATIO_MAX_(BUREAU)                                0.0
CLOSED_AMT_CREDIT_SUM_OVERDUE_TO_PROLONGED_RATIO_MIN_(BUREAU)                                0.0
CLOSED_AMT_CREDIT_SUM_LIMIT_TO_AMT_CREDIT_SUM_OVERDUE_RATIO_MEAN_(BUREAU)                    0.0
CLOSED_AMT_CREDIT_SUM_LIMIT_TO_AMT_CREDIT_SUM_OVERDUE_RATIO_MAX_(BUREAU)                     0.0
CLOSED_AMT_CREDIT_SUM_LIMIT_TO_AMT_CREDIT_SUM_OVERDUE_RATIO_MIN_(BUREAU)                     0.0
RATIO_ACTIVE_TO_CLOSED_CREDIT_DAY_OVERDUE_MIN_(BUREAU)                                       0.0
RATIO_ACTIVE_TO_CLOSED_CNT_CREDIT_PROLONG_MIN_(BUREAU)                                       0.0
RATIO_ACTIVE_TO_CLOSED_AMT_CREDIT_SUM_OVERDUE_MIN_(BUREAU)                                   0.0
RATIO_ACTIVE_TO_CLOSED_AMT_CREDIT_SUM_OVERDUE_TO_PROLONGED_RATIO_MEAN_(BUREAU)               0.0
RATIO_ACTIVE_TO_CLOSED_AMT_CRE

In [26]:
test_ids = app_test['SK_ID_CURR'].copy()
print(app_test['SK_ID_CURR'])
app_test.drop(columns=['SK_ID_CURR'], inplace=True)

0        100001
1        100005
2        100013
3        100028
4        100038
          ...  
48739    456221
48740    456222
48741    456223
48742    456224
48743    456250
Name: SK_ID_CURR, Length: 48744, dtype: int64


In [27]:
#Train y Test
X = app_train_def_7.drop(columns = ['TARGET'])
y = app_train_def_7['TARGET']


X.columns = X.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
app_test.columns = app_test.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)

In [ ]:
app_test = app_test.reindex(columns=X.columns, fill_value=0)
test_ids = app_test['SK_ID_CURR'].copy()
print(app_test.shape)
print(X.shape)

In [28]:
best_params = {
    'boosting_type': 'gbdt',  # Fijo
    'objective': 'binary',   # Fijo
    'metric': 'auc',         # Fijo
    'num_leaves': 41,        
    'learning_rate': 0.0173115 ,   
    'max_depth': -1,         # Fijo
    'bagging_fraction': 0.891186 ,
    'bagging_freq': 16,
    'feature_fraction':   0.106319 ,
    'max_bin': 300,
    'min_child_samples': 101,
    'min_child_weight':   0.0137934,
    'min_gain_to_split': 0.0269529 ,
    'reg_alpha': 0.0906638,
    'reg_lambda': 31.1379,
    'scale_pos_weight': 1,   # Fijo
    'is_unbalance': False,   # Fijo
}

final_model_lgb = LGBMClassifier(
    n_estimators=2000,
    boosting_type=best_params['boosting_type'],
    objective=best_params['objective'],
    metric=best_params['metric'],
    num_leaves=best_params['num_leaves'],
    learning_rate=best_params['learning_rate'],
    max_bin=best_params['max_bin'],
    max_depth=best_params['max_depth'],
    bagging_fraction=best_params['bagging_fraction'],
    bagging_freq=best_params['bagging_freq'],   
    feature_fraction=best_params['feature_fraction'],
    min_child_samples=best_params['min_child_samples'],
    min_child_weight=best_params['min_child_weight'],
    min_gain_to_split=best_params['min_gain_to_split'],
    reg_alpha=best_params['reg_alpha'],
    reg_lambda=best_params['reg_lambda'],
    scale_pos_weight=1,
    is_unbalance=False,
    random_state=42
)

final_model_lgb.fit(X, y)

[LightGBM] [Warning] bagging_freq is set=16, subsample_freq=0 will be ignored. Current value: bagging_freq=16
[LightGBM] [Warning] min_gain_to_split is set=0.0269529, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0269529
[LightGBM] [Warning] feature_fraction is set=0.106319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.106319
[LightGBM] [Warning] bagging_fraction is set=0.891186, subsample=1.0 will be ignored. Current value: bagging_fraction=0.891186
[LightGBM] [Warning] bagging_freq is set=16, subsample_freq=0 will be ignored. Current value: bagging_freq=16
[LightGBM] [Warning] min_gain_to_split is set=0.0269529, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0269529
[LightGBM] [Warning] feature_fraction is set=0.106319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.106319
[LightGBM] [Warning] bagging_fraction is set=0.891186, subsample=1.0 will be ignored. Current value: bagging_frac

LGBMClassifier(bagging_fraction=0.891186, bagging_freq=16,
               feature_fraction=0.106319, is_unbalance=False,
               learning_rate=0.0173115, max_bin=300, metric='auc',
               min_child_samples=101, min_child_weight=0.0137934,
               min_gain_to_split=0.0269529, n_estimators=2000, num_leaves=41,
               objective='binary', random_state=42, reg_alpha=0.0906638,
               reg_lambda=31.1379, scale_pos_weight=1)

In [29]:
y_pred_proba = final_model_lgb.predict_proba(app_test)[:, 1]
from sklearn.metrics import roc_auc_score
#auc_test = roc_auc_score(y_test, y_pred_proba)
#print(f'AUC en test: {auc_test}')

submision = pd.DataFrame({
    'SK_ID_CURR': test_ids,
    'TARGET': y_pred_proba
})
submision['SK_ID_CURR'] = test_ids.astype('int32')
submision.to_csv('mysubmission.csv', index=False)

[LightGBM] [Warning] bagging_freq is set=16, subsample_freq=0 will be ignored. Current value: bagging_freq=16
[LightGBM] [Warning] min_gain_to_split is set=0.0269529, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0269529
[LightGBM] [Warning] feature_fraction is set=0.106319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.106319
[LightGBM] [Warning] bagging_fraction is set=0.891186, subsample=1.0 will be ignored. Current value: bagging_fraction=0.891186
